In [1]:
# imports
import numpy as np
import scipy as sc
import scipy.linalg as scl
import plotly.graph_objects as go
import plotly.express as px

# Projekt 4 in Numerik 2
Janike Katter

## Aufgabe 1: QR-Verfahren

In [2]:
# Beispiel Matrizen:
A = np.matrix('5 4 2; 0 1 -1; -1 -1 3', dtype=float)
B = np.matrix('-1 1 3; 3 3 -2; -5 2 7', dtype=float)
C = np.matrix('-5 -10 -10 5; 4 16 11 -8; 12 13 8 -4; 22 48 28 -19', dtype=float)
D = np.matrix('0 0 0 1; 0 0 -1 0; 0 1 0 0; -1 0 0 0', dtype=float)

### Einfaches QR-Verfahren

In [3]:
def qr_fehler(M):
  err = 0
  mm = 0
  for i in range(len(M)):
    for j in range(len(M[0])):
      if not j >= i:
        err += abs(M[i,j])
        mm += 1

  return err/mm

In [4]:
def qr1(M, tol=1e-3, maxIter=1000000):
  AI = M
  err = qr_fehler(AI)
  errs = []
  errs.append(err)
  iteration = 0
  while (err > tol) and (iteration < maxIter):
    iteration += 1
    Q, R = scl.qr(AI)
    AI = np.matmul(R,Q)
    err = qr_fehler(AI)
    errs.append(err)
    if iteration%10 == 0:
      print(f"curr err: {err}")

  eigs = []
  for x in range(len(AI)):
    eigs.append(AI[x,x])


  return eigs, errs

In [5]:
DA, errsa = qr1(A, maxIter=1000)
print(f"Sol: {DA}, iterations: {len(errsa)}")

curr err: 0.0462019886931826
curr err: 0.013560643205910676
curr err: 0.0517316419288405
curr err: 0.002344550647827636
curr err: 0.0015296844688961284
curr err: 0.0010758914319542905
Sol: [4.0615792986173815, 3.9384207013826305, 0.9999999999999989], iterations: 64


In [9]:
scl.eigvals(A)

array([4.00000002+0.j, 3.99999998+0.j, 1.        +0.j])

In [10]:
DB, errsb = qr1(B, maxIter=1000)

curr err: 0.0957090798245661
curr err: 0.011327572144548265
curr err: 0.004799743176386571
curr err: 0.002710822667212885
curr err: 0.0017526607209213771
curr err: 0.0012290357719098655


In [11]:
DB

[3.095117395426942, 2.990612090068653, 2.914270514504402]

In [12]:
scl.eigvals(B)

array([3.00000832+1.441312e-05j, 3.00000832-1.441312e-05j,
       2.99998336+0.000000e+00j])

In [13]:
DD, errsd = qr1(D, maxIter=1000)
print(f"sol: {DD}iter: {len(errsd)}err0: {errsd[0]}")

curr err: 0.3333333333333333
curr err: 0.3333333333333333
curr err: 0.3333333333333333
curr err: 0.3333333333333333
curr err: 0.3333333333333333
curr err: 0.3333333333333333
curr err: 0.3333333333333333
curr err: 0.3333333333333333
curr err: 0.3333333333333333
curr err: 0.3333333333333333
curr err: 0.3333333333333333
curr err: 0.3333333333333333
curr err: 0.3333333333333333
curr err: 0.3333333333333333
curr err: 0.3333333333333333
curr err: 0.3333333333333333
curr err: 0.3333333333333333
curr err: 0.3333333333333333
curr err: 0.3333333333333333
curr err: 0.3333333333333333
curr err: 0.3333333333333333
curr err: 0.3333333333333333
curr err: 0.3333333333333333
curr err: 0.3333333333333333
curr err: 0.3333333333333333
curr err: 0.3333333333333333
curr err: 0.3333333333333333
curr err: 0.3333333333333333
curr err: 0.3333333333333333
curr err: 0.3333333333333333
curr err: 0.3333333333333333
curr err: 0.3333333333333333
curr err: 0.3333333333333333
curr err: 0.3333333333333333
curr err: 0.33

In [14]:
DC, errsc = qr1(C, maxIter=2)
print(f"sol: {DC}iter: {len(errsc)}err0: {errsc[0]}")

sol: [-4.999999999999995, 15.999999999999448, 8.000000000000224, -18.999999999999677]iter: 3err0: 12.666666666666666


### QR-Verfahren mit Shift

In [54]:
import scipy.optimize as sco

In [6]:
def qr_fehler2(M):
  z = len(M)-1
  err1 = 0
  err2 = 0
  m1 = 0
  m2 = 0
  for i in range(z):
    if i < z-1:
      err1 += abs(M[z,i])
      m1 += 1
      err2 += abs(M[z,i])
      err2 += abs(M[z-1,i])
      m2 += 2
    if i == z-1:
      err1 += abs(M[z,i])
      m1 += 1
  if m2 == 0:
    m2 = 1
    err2 = 10
  return (err1/m1, err2/m2)


In [7]:
def qrshiftstep(M, tol=1e-3, maxIter=10000):
  AI = M
  errs = []
  err = qr_fehler2(AI)
  errs.append(err)
  iteration = 0
  while (err[0] > tol) and (iteration < maxIter):
    s = AI[-1,-1]
    n,m = AI.shape
    IM = np.eye(n,m)*s
    DI = AI-IM
    Q, R = scl.qr(DI)
    AI = np.matmul(R,Q)+IM 
    err = qr_fehler2(AI)
    errs.append(err)
    iteration += 1

  msg = 0
  if iteration > maxIter:
    if err[1] > tol:
      msg = -1
  return AI, errs, msg

In [80]:
def shorten_matrix(M, tol=1e-3):
  n,m = M.shape
  wegrows = 0
  eigvals = []
  err_real, err_imag = qr_fehler2(M)
  if err_real < tol:
    for r in reversed(range(n)):
      kannWeg = True
      for x in range(m-1):
        if not abs(M[r,x]) < tol:
          kannWeg = False
      if kannWeg:
        wegrows += 1
        eigvals.append(M[r,r])
      else:
        break
    if wegrows == 0:
      return M, None
    else:
      nn, nm = n-wegrows, m-wegrows
      if nn == 0:
        nn = 1
      if nm == 0:
        nm = 1
      NM = M[:nn,:nm]
      return NM, eigvals
  else:
    NM = M[:n-2,:m-2]
    R = M[n-2:,m-2:]
    vorne = -(-R[0,0]-R[1,1])/(2*(R[0,1]*R[1,0]-(R[0,0]*R[1,1])))
    unterroot = (-R[0,0]-R[1,1])**2 - 4*(R[1,0]*R[0,1]-R[0,0]*R[1,1])
    hinten = sc.sqrt(unterroot)/(2*(R[0,1]*R[1,0]-(R[0,0]*R[1,1])))
    eigvals.append(vorne+hinten)
    eigvals.append(vorne-hinten)
    return NM, eigvals
  print("Error: shorten_matrix should never end up here")
  return M, eigvals

In [73]:
def qr2(M, tol=1e-3, maxIter=10000):
  n,m = M.shape
  ende = False 
  AI = M
  errors = []
  eigvals = []
  while not ende:
    AI, errs, msg = qrshiftstep(AI, tol, maxIter)
    if msg == -1:
      print('No convergence for tolerance. Now breaking')
      return 0,0
    print(f'qrshiftsteps: {AI}')
    AI, vals = shorten_matrix(AI)
    print(f'curr vals: {vals}')
    if not len(vals) > 0:
      print(f'Critical Error, no eigenvalues')
      ende = True
    else:
      print(f'curr: {AI}')
      eigvals.extend(vals)
      errors.extend(errs)
      n,m = AI.shape
      if (n <= 1) or (m <= 1):
        if (n==1) or (m == 1):
          eigvals.append(AI[0,0])
        ende = True
  return eigvals, errors


In [10]:
eig, errs = qr2(A, tol=1e-4)

qrshiftsteps: [[ 1.00000155e+00  6.00277748e-01 -4.48773049e+00]
 [ 7.74979056e-06  4.01082863e+00 -2.12125350e+00]
 [ 1.42324649e-10  5.52937691e-05  3.98916982e+00]]
curr vals: [3.989169822723613]
curr: [[1.00000155e+00 6.00277748e-01]
 [7.74979056e-06 4.01082863e+00]]
qrshiftsteps: [[1.00000155e+00 6.00277748e-01]
 [7.74979056e-06 4.01082863e+00]]
curr vals: [4.0108286321778825]
curr: [[1.00000155]]


In [11]:
eig


[3.989169822723613, 4.0108286321778825, 1.0000015450985074]

In [47]:
eigb, errsb2 = qr2(B)

qrshiftsteps: [[ 2.91448861e+00 -5.96560040e+00  6.79283436e+00]
 [ 7.46992470e-04  3.02863418e+00  1.50254614e+00]
 [ 5.22399491e-07 -8.19179490e-04  3.05687721e+00]]
curr vals: [3.0568772099307413]
curr: [[ 2.91448861e+00 -5.96560040e+00]
 [ 7.46992470e-04  3.02863418e+00]]
qrshiftsteps: [[ 2.91448861e+00 -5.96560040e+00]
 [ 7.46992470e-04  3.02863418e+00]]
curr vals: [3.028634175147]
curr: [[2.91448861]]


In [48]:
eigb

[3.0568772099307413, 3.028634175147, 2.914488614922254]

In [49]:
eigc, errsc2 = qr2(C)

qrshiftsteps: [[  7.81553625  -0.33856817  -3.56345733  63.56529567]
 [ -2.31047402   1.56301503  -4.95447509 -27.85592519]
 [  2.51646773   4.11809126  -1.68510606   4.82449498]
 [ -1.2254727    0.28081237   0.3172816   -7.69344522]]
Error: shorten_matrix should never end up here
curr vals: []
Critical Error, no eigenvalues


In [50]:
eigd, errsd2 = qr2(D)

qrshiftsteps: [[ 0.  0.  0.  1.]
 [ 0.  0. -1.  0.]
 [ 0.  1.  0.  0.]
 [-1.  0.  0.  0.]]
Error: shorten_matrix should never end up here
curr vals: []
Critical Error, no eigenvalues


### QR-Verfahren mit oberer Hessenbergform

In [12]:
import math

In [13]:
def householderspiegelung(M):
  n,m = M.shape
  vec = M[1:,0].transpose()
  housevector = vec + math.copysign(scl.norm(vec), vec[0,0])*np.eye(1, n-1)
  scal = 2/ np.dot(housevector, housevector.transpose())
  M[1,0] = math.copysign(scl.norm(vec), -1*vec[0,0])
  for i in range(2,m):
    M[i,0] = 0
  vecs = np.array([])
  for z in range(1,n):
    vector = M[1:,z].transpose()
    vector_ = vector - scal*np.dot(housevector,vector.transpose())*housevector
    #print(f"1.{z}: {vector} -> {vector_}")
    for i in range(1,n):
      M[i,z] = vector_[0,i-1]
  #print(f"Matrix nach erster Iteration: {M}")
  for r in range(0,n):
    vector = M[r,1:]
    row = vector - scal*np.dot(vector,housevector.transpose())*housevector
    #print(f'2.{r}: {vector} -> {row}')
    _,l = row.shape
    for x in range(l):
      M[r,x+1] = row[0,x]

  return M


In [14]:
def hessenberg(M):
  n,m = M.shape
  retM = M
  XM = M
  newM = M
  for x in range(n-2):
    newM = householderspiegelung(XM)
    r,c = newM.shape
    for xr in range(r):
      for yc in range(c):
        retM[x+xr,x+yc] = newM[xr,yc]
    XM = newM[1:,1:]
  return retM


In [15]:
def qr3(M, tol=1e-3, maxIter=10000):
  nM = hessenberg(M)
  eigen, errs = qr2(nM, tol, maxIter)
  return eigen, errs

In [ ]:
alpha = np.matrix('2 1 0; 3 5 -5; 4 0 0', dtype=float)
print(alpha)
hess_ = householderspiegelung(alpha)
print(hess_)

[[ 2.  1.  0.]
 [ 3.  5. -5.]
 [ 4.  0.  0.]]
1.1: [[5. 0.]] -> [[-3. -4.]]
1.2: [[-5.  0.]] -> [[3. 4.]]
Matrix nach erster Iteration: [[ 2.  1.  0.]
 [-5. -3.  3.]
 [ 0. -4.  4.]]
2.1: [[-3.  3.]] -> [[-0.6  4.2]]
2.2: [[-4.  4.]] -> [[-0.8  5.6]]
[[ 2.   1.   0. ]
 [-5.  -0.6  4.2]
 [ 0.  -0.8  5.6]]


In [ ]:
hessenberg(alpha)

1.1: [[-0.6 -0.8]] -> [[ 0.6 -0.8]]
1.2: [[4.2 5.6]] -> [[-4.2  5.6]]
Matrix nach erster Iteration: [[ 2.   1.   0. ]
 [ 5.   0.6 -4.2]
 [ 0.  -0.8  5.6]]
2.1: [[ 0.6 -4.2]] -> [[-0.6 -4.2]]
2.2: [[-0.8  5.6]] -> [[0.8 5.6]]


matrix([[ 2. ,  1. ,  0. ],
        [ 5. , -0.6, -4.2],
        [ 0. ,  0.8,  5.6]])

In [ ]:
hessenberg(np.matrix('1 2 1 2; 3 1 3 2; 6 1 2 2; 6 1 3 4', dtype=float))

1.1: [[1. 1. 1.]] -> [[-1.66666667 -0.33333333 -0.33333333]]
1.2: [[3. 2. 3.]] -> [[-4.33333333 -1.66666667 -0.66666667]]
1.3: [[2. 2. 4.]] -> [[-4.66666667 -1.33333333  0.66666667]]
Matrix nach erster Iteration: [[ 1.          2.          1.          2.        ]
 [-9.         -1.66666667 -4.33333333 -4.66666667]
 [ 0.         -0.33333333 -1.66666667 -1.33333333]
 [ 0.         -0.33333333 -0.66666667  0.66666667]]
2.1: [[-1.66666667 -4.33333333 -4.66666667]] -> [[ 6.55555556 -0.22222222 -0.55555556]]
2.2: [[-0.33333333 -1.66666667 -1.33333333]] -> [[ 2.11111111 -0.44444444 -0.11111111]]
2.3: [[-0.33333333 -0.66666667  0.66666667]] -> [[ 0.11111111 -0.44444444  0.88888889]]
1.1: [[-0.44444444 -0.44444444]] -> [[ 0.46718963 -0.42047066]]
1.2: [[-0.11111111  0.88888889]] -> [[0.06423857 0.89350016]]
Matrix nach erster Iteration: [[ 6.55555556 -0.22222222 -0.55555556]
 [-2.11403307  0.46718963  0.06423857]
 [ 0.         -0.42047066  0.89350016]]
2.1: [[0.46718963 0.06423857]] -> [[-0.46992

matrix([[ 1.        ,  2.        ,  1.        ,  2.        ],
        [-9.        ,  6.55555556, -0.22222222, -0.55555556],
        [ 0.        , -2.11403307, -0.4699202 ,  0.03959484],
        [ 0.        ,  0.        ,  0.37292818,  0.91436464]])

In [62]:
eigena, errsac = qr3(A, tol=1e-4)

qrshiftsteps: [[ 3.00641009e+00  2.85192604e+00 -2.72818069e+00]
 [-1.29002846e+00  5.15990267e+00 -2.09791396e+00]
 [ 0.00000000e+00  5.69072567e-09  8.33687249e-01]]
curr vals: [0.8336872490875357]
curr: [[ 3.00641009  2.85192604]
 [-1.29002846  5.15990267]]
qrshiftsteps: [[ 4.83370966  0.97282728]
 [-3.16912722  3.3326031 ]]
Error: shorten_matrix should never end up here
curr vals: []
Critical Error, no eigenvalues


In [ ]:
scl.eigvals(C)

array([-1.66964532e+01+0.j,  1.66964532e+01+0.j,  7.77156117e-16+5.j,
        7.77156117e-16-5.j])

In [67]:
HA = hessenberg(C)
HA

matrix([[-5.00000000e+00, -1.97027602e+00, -5.11037363e+00,
          1.39643150e+01],
        [ 2.53771551e+01,  5.00000000e+00,  7.20917209e+00,
         -6.10231848e+01],
        [ 0.00000000e+00,  1.77635684e-15, -2.91292250e+00,
          1.78241194e+01],
        [ 0.00000000e+00,  0.00000000e+00, -1.87864078e+00,
          2.91292250e+00]])

In [53]:
HE = scl.hessenberg(D)
HE

ValueError: ignored

In [39]:
Q,R = scl.qr(HE)
HE = np.matmul(R,Q)
print(HE)

[[ 4.17220408e+00  1.27096845e+00 -3.20707272e+00]
 [-2.33320074e-02  3.82779592e+00 -3.61102076e+00]
 [ 0.00000000e+00  1.12976599e-12  1.00000000e+00]]


In [74]:
eig,errs = qr2(HA)

qrshiftsteps: [[ 1.54898409e+00 -2.63058628e+01 -4.36752565e+01 -4.52256734e+01]
 [ 1.04156826e+00 -1.54898409e+00 -1.55913151e+00 -6.44971362e+00]
 [ 0.00000000e+00 -4.07164352e-15  8.41139668e+00  1.09907199e+01]
 [ 0.00000000e+00  0.00000000e+00 -8.71204022e+00 -8.41139668e+00]]
curr vals: [-0.05495510680206661, 0.7278668412759057]
curr: [[  1.54898409 -26.30586284]
 [  1.04156826  -1.54898409]]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:32: DeprecationWarning:

scipy.sqrt is deprecated and will be removed in SciPy 2.0.0, use numpy.lib.scimath.sqrt instead



qrshiftsteps: [[  2.08235263 -26.22896562]
 [  1.11846547  -2.08235263]]
curr vals: [-0.13335747674376242, 0.29994568716121756]
curr: []


In [75]:
eig

[-0.05495510680206661,
 0.7278668412759057,
 -0.13335747674376242,
 0.29994568716121756]

In [77]:
scl.eigvals(C)

array([1.27675648e-15+5.j, 1.27675648e-15-5.j, 7.21644966e-16+5.j,
       7.21644966e-16-5.j])

In [64]:
tozero(0.654505520377543+0.344944796232937j)

0.7031209046871104

In [65]:
sc.sqrt(-1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning:

scipy.sqrt is deprecated and will be removed in SciPy 2.0.0, use numpy.lib.scimath.sqrt instead



1j

In [81]:
eig, err = qr2(C)

qrshiftsteps: [[ 1.54898409e+00 -2.63058628e+01 -4.36752565e+01 -4.52256734e+01]
 [ 1.04156826e+00 -1.54898409e+00 -1.55913151e+00 -6.44971362e+00]
 [ 0.00000000e+00 -4.07164352e-15  8.41139668e+00  1.09907199e+01]
 [ 0.00000000e+00  0.00000000e+00 -8.71204022e+00 -8.41139668e+00]]
curr vals: [-0.19999999999999407, 0.20000000000000728]
curr: [[  1.54898409 -26.30586284]
 [  1.04156826  -1.54898409]]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:32: DeprecationWarning:

scipy.sqrt is deprecated and will be removed in SciPy 2.0.0, use numpy.lib.scimath.sqrt instead



qrshiftsteps: [[  2.08235263 -26.22896562]
 [  1.11846547  -2.08235263]]
curr vals: [-0.19999999999999196, 0.19999999999997758]
curr: []


In [82]:
eig

[-0.19999999999999407,
 0.20000000000000728,
 -0.19999999999999196,
 0.19999999999997758]